In [103]:
import networkx as nx

Examples of undirected graphs

In [104]:
def initialize_graph_1():
    G = nx.Graph()
    nodes = ('u', 'v', 'w', 'x', 'y', 'z')
    G.add_nodes_from(nodes)
    edges = (('u','v'), ('u','w'), ('v','w'), ('v','x'), ('w','x'),
             ('w','y'), ('y','x'), ('x','z'))
    G.add_edges_from(edges)
    return G

In [105]:
def initialize_graph_2():
    G = nx.Graph()
    nodes = ('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'L')
    G.add_nodes_from(nodes)
    edges = (('A','B'), ('A','C'), ('A','D'), ('A','E'), ('A','F'),
             ('A','H'), ('B','C'), ('C','D'), ('D','L'), ('F','G'),
             ('G','H'), ('H','I'), ('H','L'))
    G.add_edges_from(edges)
    return G

Closeness centrality algorithm for unweighted graphs

In [106]:
def closeness_centrality(G, v):
    length = dict(nx.all_pairs_shortest_path_length(G))
    sum = 0
    
    for node in G.nodes():
        sum = sum + length[v][node]
        
    return (G.number_of_nodes() - 1) / sum
    

In [107]:
G = initialize_graph_1()

for n in G.nodes():
    print(closeness_centrality(G, n))

0.5555555555555556
0.7142857142857143
0.8333333333333334
0.8333333333333334
0.625
0.5


In [108]:
G = initialize_graph_2()

for n in G.nodes():
    print(closeness_centrality(G, n))

0.75
0.47368421052631576
0.5294117647058824
0.5294117647058824
0.45
0.5
0.45
0.6428571428571429
0.4090909090909091
0.47368421052631576


Example of directed graph

In [109]:
def initialize_weighted_graph():
    G = nx.Graph()
    nodes = ('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'L')
    G.add_nodes_from(nodes)
    G.add_edge('A','B',weight=2)
    G.add_edge('A','C',weight=5)
    G.add_edge('A','D',weight=9)
    G.add_edge('A','E',weight=7)
    G.add_edge('A','F',weight=2)
    G.add_edge('B','C',weight=2)
    G.add_edge('C','D',weight=3)
    G.add_edge('D','L',weight=8)
    G.add_edge('F','G',weight=1)
    G.add_edge('G','H',weight=1)
    G.add_edge('H','I',weight=3)
    G.add_edge('H','L',weight=4)
    
    return G

Closeness centrality algorithm for weighted graphs

In [110]:
def closeness_centrality_weighted(G, v):
    length = dict(nx.all_pairs_dijkstra_path_length(G))
    sum = 0
    
    for node in G.nodes():
        sum = sum + length[v][node]
        
    return (G.number_of_nodes() - 1) / sum
    

In [111]:
G = initialize_weighted_graph()

for node in G.nodes():
    print("{}: {}".format(node, closeness_centrality_weighted(G,node)))

A: 0.20454545454545456
B: 0.17307692307692307
C: 0.14285714285714285
D: 0.1111111111111111
E: 0.09
F: 0.20454545454545456
G: 0.1956521739130435
H: 0.18
I: 0.12162162162162163
L: 0.12162162162162163


In [112]:
import numpy as np
from numpy import size
import random
import sklearn

Eppstein-Wang algorithm for weighted and unweighted graphs

In [113]:
def eppstein_wang(G,k,weight=False,verbose=False):
    # Number of nodes in the graph
    n = G.number_of_nodes()
    # Initialize the array of sum
    sum = np.zeros(n)
    # Save a list of nodes
    nodes = list(G.nodes())
    # Get k random index that refers to k nodes
    index = random.sample(range(n), k)
    
    # Create the matrix of distance between nodes
    length = dict()
    if weight == False:
        length = dict(nx.all_pairs_shortest_path_length(G))
    else:
        length = dict(nx.all_pairs_dijkstra_path_length(G))
    
    # For each node compute the distance respect the k nodes
    for i in index:
        tmp_node = nodes[i]
        for j in range(n):
            sum[j] = sum[j] + length[tmp_node][nodes[j]]
    
    # Initialize the approximated cost
    c = np.zeros(n)
    
    # Save the results in the array
    for i in range(len(sum)):
        c[i] = 1 / ((n * sum[i]) / (k * (n-1)))
        
    if verbose:
        for i in range(len(sum)):
            print("{}: {}".format(nodes[i], c[i]))
    
    return c;
    
    

In [118]:
G = initialize_graph_1()

result = eppstein_wang(G,6,verbose=True)

u: 0.5555555555555556
v: 0.7142857142857143
w: 0.8333333333333334
x: 0.8333333333333334
y: 0.625
z: 0.5


In [119]:
G = initialize_weighted_graph()

result = eppstein_wang(G,5,weight=True,verbose=True)

A: 0.1875
B: 0.1323529411764706
C: 0.10465116279069768
D: 0.08653846153846154
E: 0.1
F: 0.20454545454545453
G: 0.1956521739130435
H: 0.1875
I: 0.13636363636363638
L: 0.125


Approximating Betweenness Centralities

In [145]:
def app_betweenness_centralities(G,k,weight=False):
    # Initialize array of cost
    b = np.zeros(G.number_of_nodes())
    # Save a list of nodes
    nodes = list(G.nodes())
    
    # For each node compute the distance respect the k nodes
    for i in range(k):
        # Get random pair
        pair = random.sample(nodes,2)
        # Compute shortest path between pair and get  a random between the resulting elements
        shortest_path = random.choice(list(nx.all_shortest_paths(G,pair[0],pair[1],weight=weight)))
        
        # If the node is not in the path update the cost
        for i in range(len(nodes)):
            if nodes[i] != pair[0] and nodes[i] != pair[1]:
                b[i] = b[i] + 1/k
    return b
        

In [146]:
G = initialize_weighted_graph()

app_betweenness_centralities(G,4,True)

array([0.75, 1.  , 1.  , 0.25, 0.75, 0.75, 0.75, 1.  , 0.75, 1.  ])